In [1]:
from sdRDM.generator import generate_python_api


# generate_python_api("../specifications/model.md", "../", "pyEED")

# PYEED data model

In [1]:
%reload_ext autoreload
%autoreload 2

from pyEED.core.proteinsequence import ProteinSequence

## Query NCBI

In [2]:
aldolase = ProteinSequence.from_ncbi("XP_017143456.1")

print(aldolase)

ProteinSequence
├── id = XP_017143456.1
├── name = fructose-bisphosphate aldolase isoform X3
├── sequence = MTTYFNYPSKELQDELRGIAQAIVAPGKGILAADESVSTMGKRLQDIGVENSDENRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFVDILKKKGIIPGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPQEIGLATVLALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLGRPWALTFSYGRALQASVLKAWGGKKENIAAGQNELLKRARANALASQGKYVAGSIPSYAANTSLFIAGHKY
├── organism
│   └── Organism
│       ├── id = taxon:7229
│       ├── name = Drosophila miranda
│       └── taxonomy_id = 7229
├── regions
│   └── 0
│       └── Region
│           ├── id = region0
│           ├── start = 14
│           ├── end = 363
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │       ├── id = site0
│   │       ├── name = active
│   │       ├── type = active
│ 

## BLAST search

In [3]:
blast_result_sequences = aldolase.blast(n_hits=40)

Running blast search for fructose-bisphosphate aldolase isoform X3 from Drosophila miranda...


Fetching protein sequences: 100%|██████████| 40/40 [00:36<00:00,  1.08it/s]


In [5]:
[seq.get_nucleotide_seq() for seq in blast_result_sequences]

In [11]:
blast_result_sequences[0].get_nucleotide_seq()

In [ ]:
from IPython.display import display, HTML
import time


def animate_blinking_dots():
    for i in range(3):  # Adjust the number of dots as needed
        text = "." * (i + 1)
        display(HTML(f"<h1>{text}</h1>"))
        time.sleep(1)  # Adjust the sleep duration as needed


animate_blinking_dots()

## Storing `ProteinSequence`s in a database



In [ ]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 

>[!NOTE]
>
>If docker is not isntalled on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64 
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Connect to the database

In [ ]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

ValueError: Could not connect to database: No module named 'pymysql'

In [ ]:
# Create the tables in the database
create_tables(db_connector=db, model=ProteinSequence)

AttributeError: 'str' object has no attribute '__fields__'

### Populate the database

In [ ]:
# Transform the data into a list of dictionaries
blast_result_data = [result.dict() for result in blast_result_sequences]

# Insert the data into the database
db.connection.insert("Reactant", blast_result_data)

NameError: name 'blast_result_sequences' is not defined